In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taxonomy-dataset/16S_sequences.fasta


In [2]:
fasta_path = "/kaggle/input/taxonomy-dataset/16S_sequences.fasta"

sequences = []
seq_records = set()


def read_from_file(fasta_path, min_length, remove_duplicates):
    seq = ""
    header = ""
    with open(fasta_path, "r") as f:
        for line in f:
            if line.startswith(">"):
                header = line[1:].strip()
                if seq:
                    seq = seq.upper()
                    seq = seq.replace("N", "")
                    if len(seq) >= min_length:
                        if remove_duplicates:
                            if seq not in seq_records:
                                seq_records.add(seq)
                                specie_id = header[3:].split()[0]
                                specie_id = specie_id[:-2]
                                species_name = " ".join(header.split()[1:3])
                                sequences.append((specie_id, species_name, seq))
                            else:
                                sequences.append((specie_id, species_name, seq))
                        seq = ""
                    header = ""
            else:
                seq += line.strip()
    return sequences

data = read_from_file(fasta_path, 100, True)

In [3]:
df = pd.DataFrame(data, columns=['label_id', 'label', 'sequence'])
df.head()

,label_id,label,sequence
0,118899,Actinomyces bovis,GGTCTATACCGGATATAACAACTCATGGCATGGTTGGTAGTGGAAA...
1,074334,Archaeoglobus fulgidus,GGGTGAGTAACACGTGAGTAACCTGCCCCACTTCTGGATAACCGCT...
2,118890,Actinokineospora fastidiosa,ATTCTGGTTGATCCTGCCAGAGGCCGCTGCTATCCGGCTGGGACTA...
3,044838,Atopobium minutum,TACTTTGGGATAAGCCTGGGAAACTGGGTCTATACCGGATATGACA...
4,118908,Amycolatopsis methanolica,TTGAACGGAGAGTTCGACCTGGCTCAGGATGAACGCTGGCGGCGCG...


In [4]:
from sklearn.model_selection import train_test_split

labels = df['label'].unique().tolist()
label2id = {"label":i for i in range(len(labels))}
df["label_id"] = df["label"].map(label2id)
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,label_id,label,sequence
0,NaN,Actinomyces bovis,GGTCTATACCGGATATAACAACTCATGGCATGGTTGGTAGTGGAAA...
1,NaN,Archaeoglobus fulgidus,GGGTGAGTAACACGTGAGTAACCTGCCCCACTTCTGGATAACCGCT...
2,NaN,Actinokineospora fastidiosa,ATTCTGGTTGATCCTGCCAGAGGCCGCTGCTATCCGGCTGGGACTA...
3,NaN,Atopobium minutum,TACTTTGGGATAAGCCTGGGAAACTGGGTCTATACCGGATATGACA...
4,NaN,Amycolatopsis methanolica,TTGAACGGAGAGTTCGACCTGGCTCAGGATGAACGCTGGCGGCGCG...
...,...,...,...
27348,NaN,Hongsoonwoonella zoysiae,GAAGGAGAAGCACCTTTCGGGGTGTGGACACTGGCGCACGGGTGAG...
27349,NaN,Insulibacter thermoxylanivorax,TGGCGGCAGGCCTAACACATGCAAGTCGAACGGACCCTTCGGGGTT...
27350,NaN,Haloarcula amylovorans,CTAATACATGCAAGTCGAGCGGAGTCATCCGGCTGAAGTCTTCGGA...
27351,NaN,Wocania indica,TTGATCCTGCCGGAGGCCATTGCTATCGGAGTCCGATTTAGCCATG...
